# Pregunta 1, parte 1

## Importacion

Se importa el módulo `pandas`, y se importa un módulo personal llamado `paths`, que contiene los paths que utilizaremos en este análisis.

In [1]:
import pandas as pd

from paths import *

Esta es la documentación de `paths` (decomentar para verlas).

In [ ]:
# Documentación del módulo 'paths'.
# import paths
# help(paths)

## Carga de datos

### Creación de la tabla mezclada

Cargar todos los .csv en dos listas distinguidas por la aparición de `furnished` en el nombre.

In [2]:
# Lista de DataFrames (Sin 'furnished' en el nombre)
L_DF_a = [pd.read_csv(dict_csv_mc_a[wNN]) for wNN in L_WNN]

# Lista de DataFrames (Con 'furnished' en el nombre)
L_DF_f = [pd.read_csv(dict_csv_mc_f[wNN]) for wNN in L_WNN]

Se unen los `DataFrames` de las listas en un único `DataFrame`, luego se crea una nueva columna llamada `'furnished'` y finalmente se unen los dos `DataFrames` en uno, con la nueva columna incluida.

In [3]:
# se unen todos los data_frame en cada lista con el comando concat
DF_a = pd.concat(L_DF_a)
DF_f = pd.concat(L_DF_f)

# Se eliminan las filas duplicadas de cada data frame por separado
DF_a.drop_duplicates(subset=DF_a.columns, ignore_index=True, inplace=True, keep='last')
DF_f.drop_duplicates(subset=DF_f.columns, ignore_index=True, inplace=True, keep='last')

# Se crea una columna con tantos ceros como la cantidad de filas de DF_a
# y con tantos unos como la cantidad de filas de DF_f 
furnished_col = pd.DataFrame(data={'furnished':[0]*(DF_a.shape[0]) + [1]*(DF_f.shape[0])})

# Concateno los data frames DF_a y DF_f y le agrego al final la columna furnished_col
df = pd.concat([DF_a, DF_f], ignore_index=True) 
df = pd.concat([df, furnished_col], axis=1)

In [4]:
# vemos cuantas filas tiene cada data frame (para reportar si hay filas en furnished 
# que no estan en all)
# print(DF_a.shape[0])
# print(DF_f.shape[0])
# nota: DF_a tiene 16295 filas y DF_f tiene 2099 filas  

Para terminar, se quitan las filas duplicadas en ambos grupos, furnished y all

In [5]:
# Quitamos las filas que sean iguales en todas las columnas con excepcion de
# la columna furnished y reindexamos
df.drop_duplicates(subset=df.columns[:-1], ignore_index=True, inplace=True, keep='last')

# comentario 1: si hay filas duplicadas con distintos valores de furnished, se eliminan las primera
# comentario 2: el comentario 1 asegura que si hay dos filas iguales con distintos valores de furnished,
# se elimine la que tiene el valor 0 para furnished, lo que tiene sentido dado que furnished puede estar
# contenido en all

In [6]:
# reportar si Reporte si existen observaciones de archivos con texto ’furnished’ que no estén
# contenidos en archivos con texto ’all’
# print(df[df['furnished'] == 0].shape[0])
# print(df[df['furnished'] == 1].shape[0])
# como ambas selecciones de filas en el dataframe tienen las mismas filas que los originales
# luego de eliminar las filas iguales, se conluye que no hay filas en furnished que no esten
# en all

## Pregunta 1, parte 2

### a) Limpieza de datos

Se crea un diccionario llamado `valores_nulos` que contendrá el nombre de las columnas de `df` como llave, y el valor de la llave será lo que reemplazará el valor `NaN`.

In [7]:
# Creamos valores a reemplazar
valores_nulos = {
#     'property_type|rent_type|location':'Sin nombre',
    'price':'$0',
    'n_rooms':'NA',
    'n_bath':'NA',
    'surface':'0.0 m2',
#     'details':'',
#     'url':'',
#     'metrocuadrado_index':0.0,
#     'furnished':0
}

In [8]:
# Reemplazamos los valores
df.fillna(value=valores_nulos, inplace=True)

#### Reemplazo de los tipos de algunas columnas

Además, en esta etapa se cambiará el tipo de algunas columnas.

##### Funciones auxiliares

In [9]:
# Función que mapeará la columna `price`
def precio_to_int(s:str)->int:
    """
    Toma un precio en forma de `str`, en el formato `'$XXX.XXX.XXX'` 
    con X algún número, y retorna el número entero de ese precio.
    """
    return int(s.replace('$', '').replace('.', ''))

# Función que mapeará la columna `surface`
def n_surface(s:str)->float:
    """
    Toma un `str` en formato "XX.Xm2" y retorna el número real con los metros cuadrados.
    """
    return float(s.replace('m2', ''))

##### Reemplazo de columnas

In [10]:
# Columna de precios
precios_col = df['price'].map(precio_to_int)
# Columna de superficie
superficie_col = df['surface'].map(n_surface)

df['price'] = precios_col
df['surface'] = superficie_col
df.rename(columns={
    'price':'price_$',
    'surface':'surface_m2'
}, inplace=True)

### b) Obtención de 3 columnas a partir de `'property_type|rent_type|location'`

`lower_case_col` es una columna con todas las palabras en minusculas.

In [11]:
lower_case_col = df['property_type|rent_type|location'].str.lower()

#### Columna `tipo de inmueble`

In [12]:
# `casa_bool_col` y `apart_bool_col` son columnas que tienen un -1 si la palabra no está 
# y un número entero positivo si la palabra está.
casa_bool_col = lower_case_col.str.find('casa')
apart_bool_col = lower_case_col.str.find('apartamento')

# Se crea una lista con la infomacion del tipo de inmueble recorriendo las dos columnas anteriores
tipo_inm_list = []
for index in range(len(lower_case_col)):
    if casa_bool_col[index] >= 0:
        tipo_inm_list.append('Casa')
    elif apart_bool_col[index] >= 0:
        tipo_inm_list.append('Apartamento')
    else:
        tipo_inm_list.append('No hay info')

# columna de tipo de inmueble
tipo_inm_col = pd.DataFrame(data = {'tipo_de_inmueble': tipo_inm_list})

# Nota: no hay columnas que no tengan una de las dos informaciones

#### Columna `Tipo de oferta`



In [13]:
# `vent_bool_col` y `arr_bool_col` son columnas que tienen un -1 si la palabra no está 
# y un número entero positivo si la palabra está.
vent_bool_col = lower_case_col.str.find('venta')
arr_bool_col = lower_case_col.str.find('arriendo')

# Se crea una lista con la infomacion del tipo de oferta recorriendo las dos columnas anteriores
tipo_ofer_list = []
for index in range(len(lower_case_col)):
    if (vent_bool_col[index] >= 0) and (arr_bool_col[index] >= 0):
        tipo_ofer_list.append('Arriendo y venta')
    elif arr_bool_col[index] >= 0:
        tipo_ofer_list.append('Arriendo')
    else:
        tipo_ofer_list.append('No hay info')

# columna de tipo de oferta
tipo_ofer_col = pd.DataFrame(data = {'tipo_de_oferta': tipo_ofer_list})

# Nota: no hay columnas que no tengan una de las dos informaciones

#### Columna `location`



In [14]:
# `sep_comas` es una serie que contiene en cada elemento una lista del string correspondiente 
# que separa por comas los grupos de strings
sep_comas = df['property_type|rent_type|location'].str.split(pat = ",")

# Se crea una lista con la infomacion location recorriendo los segundos elementos de la columna anterior
# se asume que la direccion esta al final de cada string
location_list = []
for index in range(len(sep_comas)):
    lista_strings = sep_comas[index]
    location_list.append(lista_strings[1]) 
    
# columna location
location_col = pd.DataFrame(data = {'location': location_list})

#### Agregar las 3 columnas anteriores a `df`

In [15]:
df = pd.concat([df,tipo_inm_col], axis=1)
df = pd.concat([df,tipo_ofer_col], axis=1)
df = pd.concat([df,location_col], axis=1)

In [ ]:
# prueba de que location_col no tiene el string de arriendo (solo tiene direcciones)
# prueba_0 = location_col['location'].str.lower()
# prueba_1 = prueba_0.str.find('arriendo')
# for index in range(len(prueba_1)):
#     if prueba_1[index] >= 0:
#         print('error')

## Pregunta 1, parte 3

### Esquema para agregar las columnas `price per m2` y `garajes`

* Se desarrollarán funciones que procesen las columnas `url`.
* Se crearán nuevas columnas a partir de las columnas mapeadas a partir de las funciones creadas.
* Se agregarán estas columnas a `df`.

#### Funciones

Se detallan las funciones que mapearán las columnas a continuación:

In [16]:
# Función que mapeará la columna `url`
def n_garaje(s:str)->str: 
    """
    Función que toma una url en forma de `str` y retorna un `str`, que será
    el número de garajes para la vivienda de esa url.
    """
    i_garaje = s.find('-garajes')
    
    # No se encuentra el garaje
    if i_garaje is -1: 
        return '0'
    
    # rescata el str con el número. Puede ser un número de la forma '4+'
    s_hasta_garaje = s[:i_garaje] # Se recortará el str hasta la aparición de '-garajes'
    i_garaje_ = s_hasta_garaje.rfind('-') + 1 # Se encontrará el índice siguiente de un guión '-'
    
    return s_hasta_garaje[i_garaje_:] # Se entrega el resto después de ese índice

#### Procesamiento de columnas

Se crearán las columnas y se procesarán para ser entregadas.

In [17]:
#columna de los garajes
garajes_col = df['url'].map(n_garaje)

# Columna producto de la división entre las columnas de precio y superficie
p_por_s_col = (df['price_$'] / df['surface_m2']).map(lambda x: round(x, 2))

#### Rotulación de columnas y añadirlas a `df`

In [18]:
df_ppm_and_g = pd.DataFrame(
    data={
        'price_per_m2_$/m2':p_por_s_col,
        'garajes':garajes_col
    }
)

df = pd.concat([df, df_ppm_and_g], axis=1)

## Pregunta 1, parte 4

### Esquema para clasificar:

In [19]:
# crear las 8 clasificaciones con el comando query, ademas, se crean columnmas con enteros 
# que representan dicha clasificacion

expr_base = 'tipo_de_inmueble == "{tipo_inmueble}" ' \
          + 'and surface_m2 >= {cota_inf} ' \
          + 'and surface_m2 {desig} {cota_sup}'

L_inmuebles = ['Casa']*5 + ['Apartamento']*3
bnds_sym_r = ['<']*4 + ['<='] \
           + ['<']*2 + ['<=']

L_bnds_casa = list(map(str, [80, 120, 180, 240, 360, 460]))
L_bnds_apart = list(map(str, [40, 60, 80, 120]))

bnds = [(L_bnds_casa[i], L_bnds_casa[i+1]) for i in range(len(L_bnds_casa)-1)] \
     + [(L_bnds_apart[i], L_bnds_apart[i+1]) for i in range(len(L_bnds_apart)-1)]

L_expr = [expr_base.format(
    tipo_inmueble=L_inmuebles[i], 
    cota_inf=bnds[i][0], 
    desig=bnds_sym_r[i], 
    cota_sup=bnds[i][1]
) for i in range(len(bnds))]

In [20]:
L_prod = []
tipo_prod_list = []

for i in range(len(L_expr)):
    prod = df.query(L_expr[i], inplace=False)
    list_prod = [i+1]*(prod.shape[0])
    
    L_prod.append(prod)
    tipo_prod_list += list_prod

# concateno todos los productos reseteando los indices
df = pd.concat(L_prod, ignore_index=True) 
# creo la columna con el tipo de producto
tipo_prod = pd.DataFrame(data={'Tipo_de_prod': tipo_prod_list})
# agrego la columna de tipo de producto
df = pd.concat([df, tipo_prod], axis=1)

In [ ]:
# # crear las 8 clasificaciones con el comando query, ademas, se crean columnmas con enteros 
# # que representan dicha clasificacion

# # Tipo de producto 1
# prod_1 = df.query('tipo_de_inmueble == "Casa" and surface_m2 >= 80 and surface_m2 < 120', inplace = False)
# list_prod_1 = [1]*(prod_1.shape[0])
# # Tipo de producto 2
# prod_2 = df.query('tipo_de_inmueble == "Casa" and surface_m2 >= 120 and surface_m2 < 180', inplace = False)
# list_prod_2 = [2]*(prod_2.shape[0])
# # Tipo de producto 3
# prod_3 = df.query('tipo_de_inmueble == "Casa" and surface_m2 >= 180 and surface_m2 < 240', inplace = False)
# list_prod_3 = [3]*(prod_3.shape[0])
# # Tipo de producto 4
# prod_4 = df.query('tipo_de_inmueble == "Casa" and surface_m2 >= 240 and surface_m2 < 360', inplace = False)
# list_prod_4 = [4]*(prod_4.shape[0])
# # Tipo de producto 5
# prod_5 = df.query('tipo_de_inmueble == "Casa" and surface_m2 >= 360 and surface_m2 <= 460', inplace = False)
# list_prod_5 = [5]*(prod_5.shape[0])
# # Tipo de producto 6
# prod_6 = df.query('tipo_de_inmueble == "Apartamento" and surface_m2 >= 40 and surface_m2 < 60', inplace = False)
# list_prod_6 = [6]*(prod_6.shape[0])
# # Tipo de producto 7
# prod_7 = df.query('tipo_de_inmueble == "Apartamento" and surface_m2 >= 60 and surface_m2 < 80', inplace = False)
# list_prod_7 = [7]*(prod_7.shape[0])
# # Tipo de producto 8
# prod_8 = df.query('tipo_de_inmueble == "Apartamento" and surface_m2 >= 80 and surface_m2 <= 120', inplace = False)
# list_prod_8 = [8]*(prod_8.shape[0])

In [ ]:
# # concateno todos los productos reseteando los indices
# df = pd.concat([prod_1, prod_2, prod_3, prod_4, prod_5, prod_6, prod_7, prod_8], ignore_index=True) 
# # creo la columna con el tipo de producto
# tipo_prod_list = list_prod_1 + list_prod_2 + list_prod_3 + list_prod_4 + list_prod_5 + list_prod_6 + list_prod_7 +list_prod_8 
# tipo_prod = pd.DataFrame(data = {'Tipo_de_prod': tipo_prod_list})
# # agrego la columna de tipo de producto
# df = pd.concat([df, tipo_prod], axis=1)

In [21]:
df

,property_type|rent_type|location,price_$,n_rooms,n_bath,surface_m2,details,url,metrocuadrado_index,furnished,tipo_de_inmueble,tipo_de_oferta,location,price_per_m2_$/m2,garajes,Tipo_de_prod
0,"Casa en Arriendo, VILLA DE LOS ALPES II SECTOR...",750000,3,1,90.0,CASA DE TRES NIVELES TRES ALCOBAS SALA COMEDOR...,https://www.metrocuadrado.com/inmueble/arriend...,12.2,0,Casa,Arriendo,VILLA DE LOS ALPES II SECTOR Bogotá D.C..,8333.33,0,1
1,"Casa en Arriendo, SANTA ANA ORIENTAL Bogotá D.C..",2700000,2,2,90.0,La ubicación exacta de la propiedad no se publ...,https://www.metrocuadrado.com/inmueble/arriend...,55.9,0,Casa,Arriendo,SANTA ANA ORIENTAL Bogotá D.C..,30000.00,1,1
2,"Casa en Arriendo, ESTACION VICTORIA Bogotá D.C..",1500000,3,2,90.0,"Excelente Jardin, Parqueadero interno a la cas...",https://www.metrocuadrado.com/inmueble/arriend...,28.7,0,Casa,Arriendo,ESTACION VICTORIA Bogotá D.C..,16666.67,1,1
3,"Casa en Arriendo, AUTOPISTA MUZU Bogotá D.C..",1300000,4,2,90.0,"Entrada con puerta metalica, sala comedor, coc...",https://www.metrocuadrado.com/inmueble/arriend...,38.8,0,Casa,Arriendo,AUTOPISTA MUZU Bogotá D.C..,14444.44,1,1
4,"Casa en Arriendo, PORTALES DEL NORTE Bogotá D.C..",1325000,3,3,92.0,Casa en arriendo de tres niveles cerca a la Av...,https://www.metrocuadrado.com/inmueble/arriend...,50.0,0,Casa,Arriendo,PORTALES DEL NORTE Bogotá D.C..,14402.17,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15626,"Apartamento en Arriendo, LA CABRERA Bogotá D.C..",5000000,2,2,115.0,"Lindo apartamento amoblado, ubicado en un sect...",https://www.metrocuadrado.com/inmueble/arriend...,73.7,1,Apartamento,Arriendo,LA CABRERA Bogotá D.C..,43478.26,1,8
15627,"Apartamento en Arriendo, SANTA BARBARA OCCIDEN...",4390000,2,3,110.0,amoblado con terraza.,https://www.metrocuadrado.com/inmueble/arriend...,64.2,1,Apartamento,Arriendo,SANTA BARBARA OCCIDENTAL Bogotá D.C..,39909.09,2,8
15628,"Apartamento en Arriendo, ANTIGUO COUNTRY Bogot...",4500000,2,3,120.0,"Lindo apto en el sector de Chico Virrey, cerca...",https://www.metrocuadrado.com/inmueble/arriend...,55.4,1,Apartamento,Arriendo,ANTIGUO COUNTRY Bogotá D.C..,37500.00,2,8
15629,"Apartamento en Arriendo, MOLINOS NORTE Bogotá ...",4100000,2,3,114.0,"Hermoso apto, rodeado de verde por todos lados...",https://www.metrocuadrado.com/inmueble/arriend...,59.1,1,Apartamento,Arriendo,MOLINOS NORTE Bogotá D.C..,35964.91,2,8


## Pregunta 1, parte 5

In [40]:
# packete que permite controlar el directorio de trabajo 
import os
# path del directorio de trabajo 
cwd = os.getcwd()
# cargar data
df_upz = pd.read_csv(cwd[:-7]+'data\\asignacion_upz\\barrio-upz.csv')

In [82]:
# se crea la columna location que es la columna 'location' de df, a la que 
# se le cambian todas las letras por minusculas y se le hace un split 
# para ver las palabras enlistadas   
location = df['location'].str.lower()
location = location.str.split(pat = " ")

# se crea la columna pro_location que es la columna 'pro_location' de df_upz, a la que 
# se le cambian todas las letras por minusculas y se le hace un split 
# para ver las palabras enlistadas   
pro_location = df_upz['pro_location'].str.lower()
pro_location = pro_location.str.split(pat = " ")

In [139]:
# Se crea una lista con los indices de df_upz que corresponden a cada fila de df
# en caso de que no se encuentre ningun representante en df_upz, se pone un -1
# Se usa el hecho de que uan fila en df se puede asignar a lo mas a una fila en upz 
# (algoritmo celda siguiente)
rep_list = []

# Se elige un elemento (lista por el split) de location
for name_df_completo in location:
    # Se acorta para sacar el primer elemento (['']) y el los dos ultimos (['Bogota', 'D.C..'])
    name_df = name_df_completo[1:-2] 
    # se fija un indice inicial para iterar sobre todos los indices de df_upz hasta encontrar
    # un representante
    indice_upz = 0  
    index_encontrado = False 
    while not(index_encontrado) and indice_upz < len(pro_location):
        # nos aseguramos que es una lista desde el split y no un 'nan'
        if type(pro_location[indice_upz]) == list:
            # vemos si pro_location[indice_upz] es el representante 
            if pro_location[indice_upz] == name_df:
                # en este caso paramos el loop
                index_encontrado = True 
        # sino, se continua con el otro indice 
        else:
            indice_upz = indice_upz + 1 
    # Luego terminar el loop, tenemos dos casos
    # se encontro un indice
    if index_encontrado:
        rep_list.append(indice_upz)
    else:
        rep_list.append(-1)

15000.0

In [99]:
pro_location = df_upz['pro_location'].str.lower()
pro_location_0 = pro_location.str.split(pat = " ")
print(pro_location_0)

0             [palermo]
1          [belalcazar]
2               [sears]
3            [galerias]
4      [banco, central]
             ...       
463       [nuevo, muzu]
464        [el, condor]
465          [laguneta]
466      [villa, maria]
467                 NaN
Name: pro_location, Length: 468, dtype: object


In [135]:
buenos = 0
malos = 0
malos_out = 0
for name_df in columnita_0:
    name_df_cut = name_df[1:-2] 
    cont = 0
    for indice in range(len(pro_location_0)):
        if type(pro_location_0[indice]) == list:
            if pro_location_0[indice] == name_df_cut:
                cont = cont + 1
    if cont == 0:
        malos = malos + 1
    elif cont > 1:
        malos_out = malos_out + 1
    else:
        buenos = buenos + 1

In [136]:
print((malos/(buenos+malos))*100)
print(malos_out)

10.958991747169087
0


In [111]:
type(pro_location_0[0]) is list

True

In [113]:
for i in range(60):
    print(columnita_0[i][1:-2])

['villa', 'de', 'los', 'alpes', 'ii', 'sector']
['santa', 'ana', 'oriental']
['estacion', 'victoria']
['autopista', 'muzu']
['portales', 'del', 'norte']
['santa', 'ana', 'occidental']
['marruecos', 'ii', 'sector']
['santa', 'barbara']
['tunjuelito']
['gilmar']
['guiparma']
['valladolid']
['el', 'rincon', 'de', 'los', 'angeles']
['la', 'chucua']
['predio', 'caldas']
['ciudad', 'jardin', 'norte']
['las', 'flores']
['prado', 'pinzon']
['juan', 'xxiii']
['maranta', 'iii', 'sector']
['centro', 'americas']
['el', 'contador']
['balcones', 'de', 'oriente']
['villa', 'del', 'rio']
['rincon', 'del', 'chico']
['santa', 'barbara', 'oriental']
['san', 'cipriano']
['mazuren']
['margaritas', 'i', 'sector']
['urbanizacion', 'tibabuyes', 'universal']
['villa', 'del', 'rio']
['urbanizacion', 'el', 'cortijo']
['caobos', 'salazar']
['bosque', 'de', 'pinos']
['bosque', 'de', 'pinos']
['campin', 'occidental']
['veracruz']
['usaquen']
['americas', 'occidental']
['villa', 'sagrario']
['capellania']
['san', 'j

In [77]:
cont = 0
for index in range(df.shape[0]):
    # Tomo el nombre de la locacion, le saco los ultimos 13 caracteres que dicen Bogotá D.C..
    # Luego pongo todas las letras en minusculas y le llamo a la variable name_location 
    name_location = (df['location'][index][:-13]).lower()
    # Busco el name_location en la columna 'pro_location' del data-frame df_upz
    sub_df_upz = pro_location[pro_location == name_location]
    # printeo si hay mas de una direccion
    if sub_df_upz.shape[0] == 0:      
        print(name_location)
        cont += 1 

 villa de los alpes ii sector
 santa ana oriental
 estacion victoria
 autopista muzu
 portales del norte
 santa ana occidental
 marruecos ii sector
 santa barbara
 tunjuelito
 gilmar
 guiparma
 valladolid
 el rincon de los angeles
 la chucua
 predio caldas
 ciudad jardin norte
 las flores
 prado pinzon
 juan xxiii
 maranta iii sector
 centro americas
 el contador
 balcones de oriente
 villa del rio
 rincon del chico
 santa barbara oriental
 san cipriano
 mazuren
 margaritas i sector
 urbanizacion tibabuyes universal
 villa del rio
 urbanizacion el cortijo
 caobos salazar
 bosque de pinos
 bosque de pinos
 campin occidental
 veracruz
 usaquen
 americas occidental
 villa sagrario
 capellania
 san jorge
 britalia
 portales del norte
 cantalejo
 predio caldas
 santa teresa
 cedro bolivar
 salitre i
 el mortiño
 villa del prado
 portales del norte
 portales del norte
 gilmar
 pradera norte
 la macarena
 mandalay ii
 ciudad techo
 villa rios
 el rincon de los angeles
 urbanizacion carvajal
 

 santa barbara oriental
 lourdes
 los cedros oriental
 casa blanca suba
 san francisco
 niza suba
 santa helena i
 la castellana
 niza suba
 santa barbara alta
 mochuelo norte
 la liberia
 las orquideas
 la nueva gloria
 las margaritas
 cedro narvaez
 club los lagartos
 san antonio nor - occidental
 banco central
 caobos salazar
 conejera
 san jose de bavaria
 casa blanca suba
 margaritas i sector
 normandia
 giralda
 julio florez
 urbanizacion avenida decima

 san antonio nor - occidental
 atenas
 cedritos
 zona industrial
 atenas
 villa del prado
 lisboa
 el batan

 prado veraniego norte
 santa bibiana
 margaritas i sector
 la liberia
 casa blanca suba
 casa blanca suba
 rafael uribe
 cedritos
 normandia occidental
 casa blanca suba
 villa teresita - santa lucia
 villas del mediterraneo
 villas del mediterraneo
 casa blanca suba
 estoril
 cedro narvaez
 la liberia
 acacias usaquen
 santa barbara occidental
 acacias usaquen
 bosque popular
 cedritos
 puente largo
 el encanto
 casa bla

 ciudad jardin sur
 monaco
 rincon del chico
 polo club
 las margaritas
 normandia
 bonanza
 villa del prado
 potosi
 villa magdala
 la salle
 sabanalarga
 conejera
 normandia occidental
 rionegro
 la esmeralda
 normandia
 el plan
 monaco
 los naranjos
 casa blanca
 san fernando
 ciudad jardin sur
 niza norte
 urbanizacion santa coloma
 popular modelo
 santa cecilia
 san jose de bavaria
 santa rosa
 cerros de santa barbara
 santa ana oriental
 casa blanca
 normandia
 atenas
 cerros de santa barbara
 santa barbara alta
 santa ana oriental
 club los lagartos
 estoril
 babaria techo iii
 estrella del norte
 la esmeralda
 la castellana
 polo club
 cerros de santa barbara
 modelia
 cerros de santa barbara

 polo club
 potosi
 lisboa
 el batan
 rincon del chico

 el batan
 urbanizacion bosques de medina
 cerros de santa barbara
 usaquen
 centro administrativo
 el batan
 monaco
 la calleja
 santa cecilia
 suba naranjos
 el contador
 bella suiza
 prado veraniego
 santa barbara alta
 chico nort

 armenia
 palermo
 torca
 torca
 niza suba
 santa barbara central
 las orquideas
 san diego
 el contador
 casa blanca
 casa blanca
 santa ana oriental
 la cabaña
 niza norte
 concepcion norte
 campin occidental
 normandia
 potosi
 niza
 san cayetano
 san joaquin
 santa cecilia
 la calleja
 san jose
 la carolina
 quinta camacho
 los andes
 la castellana
 emaus
 monaco
 normandia
 puente largo
 estoril
 chico norte ii
 niza norte
 santa ana occidental
 santa ana oriental
 la calleja
 prados del salitre
 pasadena
 el salitre
 tierra linda
 niza norte
 quinta camacho
 niza suba
 puente largo
 caobos salazar
 casa blanca
 niza suba
 san luis
 monaco
 casa blanca suba
 santa ana occidental
 once de noviembre
 chico norte ii
 palermo
 garces navas oriental
 estrella
 santa isabel sur
 polo club
 los andes norte
 chapinero sur occidental
 los cedritos
 los andes
 niza norte
 usaquen
 los andes
 la castellana
 campin occidental
 niza suba
 villa magdala
 alfonso lopez
 chico norte ii
 san jose 

 casa blanca
 santa ana oriental
 potosi
 belalcazar
 santa rosa
 las villas
 rincon del chico
 santa barbara alta
 santa barbara occidental
 los cedritos
 santa ana occidental
 santa ana occidental
 casa blanca
 santa ana oriental
 niza
 bosque de pinos
 estoril
 santa ana oriental
 chapinero norte
 lago gaitan
 la capuchina
 el refugio
 santa ana occidental
 los andes
 los cedritos
 los cedritos
 los cedritos
 casa blanca
 san jose de bavaria
 chico norte ii
 santa ana oriental
 la esperanza
 el recuerdo
 san jose de bavaria
 sierras de moral
 casa blanca
 bosque de pinos
 el refugio
 santa ana oriental
 la castellana
 san jose de bavaria
 sierras del moral
 suba naranjos
 san jose de bavaria
 santa bibiana
 quinta camacho
 san jose de bavaria
 la castellana
 los cedritos
 quinta paredes
 alfonso lopez
 la soledad
 santa ana oriental
 estoril
 usaquen
 torca
 cedro bolivar
 niza suba
 el batan
 los andes
 los andes
 los andes
 casa blanca
 casa blanca
 quinta camacho
 conejera
 san j

...


 chico norte iii
 plenitud
 milenta
 santa isabel sur
 el nogal
 antiguo country
 antiguo country
 chico
 iberia
 bochica

 alcala
 chico norte iii
 santa barbara central
 chico norte iii
 monaco
 santa barbara occidental
 santa bibiana
 pardo rubio
 puente largo
 contador
 santa bibiana
 santa bibiana
 santa barbara occidental
 san patricio
 chico
 chico norte iii
 estoril
 molinos norte
 santa barbara central
 campo alegre
 chico
 santa bibiana
 los cedritos
 centro administrativo
 las acacias
 san patricio
 primavera occidental
 chico norte ii
 chico norte
 chico norte ii
 chico norte
 la salle
 rincon del chico
 bosque calderon
 chico norte
 molinos norte
 bellavista
 chico norte iii
 santa barbara occidental
 bochica
 san patricio
 los rosales
 chico norte ii
 chico norte
 molinos norte
 santa bibiana
 cedro salazar
 santa barbara central
 santa barbara oriental
 la carolina
 chico norte iii
 el refugio
 chico norte
 chico norte iii
 san patricio
 antiguo country
 santa bibiana
 

 santa barbara occidental
 san patricio
 jazmin
 santa bibiana
 santa barbara occidental
 chico norte iii
 chico
 los rosales
 santa barbara central
 santa barbara oriental
 la calleja
 antiguo country
 bellavista
 la cabrera
 bellavista
 chico norte ii
 molinos norte
 country club
 la salle
 florida blanca
 santa barbara central
 chico
 emaus
 cedro narvaez
 la catedral
 antiguo country
 santa ana occidental
 pasadena
 santa bibiana
 las acacias
 santa ana occidental
 chico
 chico
 bellavista
 emaus
 la carolina
 santa isabel sur
 florida blanca
 pardo rubio
 el contador
 rincon del chico
 san patricio
 chico
 ciudad salitre nor-oriental
 santa ana oriental
 santa bibiana
 country club
 molinos norte
 santa barbara occidental
 chico norte iii
 santa barbara occidental
 los cedritos
 el batan
 rionegro
 campo alegre
 cedritos
 san patricio
 chico
 samper
 san patricio
 contador
 bella suiza
 santa bibiana
 chico norte
 molinos norte
 molinos norte
 chico norte
 santa bibiana
 monaco
 e

 santa barbara central

 santa bibiana
 santa bibiana
 chico norte iii
 molinos norte
 chico norte iii
 molinos norte
 santa barbara central
 los ejidos
 los ejidos

 chico norte iii
 molinos norte
 chico
 chico norte iii
 usaquen
 chico norte ii
 san patricio
 monaco
 estoril
 antiguo country

 chico norte ii
 molinos norte
 puente largo
 chico norte
 chico norte ii
 chico norte
 santa barbara central
 los rosales
 rincon de santa paula
 san patricio
 santa barbara occidental
 chico norte
 puente largo
 chico norte
 santa bibiana
 los cedros oriental
 san patricio
 chico norte ii
 santa barbara alta
 santa barbara central
 antiguo country
 chico norte ii
 el refugio
 chico
 santa bibiana
 chico norte iii
 sabanalarga
 santa barbara occidental
 bosque calderon tejada
 santa barbara occidental
 chico norte
 la esperanza norte
 santa bibiana
 santa bibiana
 san patricio
 antiguo country
 el nogal
 el nogal
 molinos norte
 san patricio
 santa bibiana
 rincon del chico
 san patricio
 santa


 san patricio
 emaus
 santa bibiana
 chico norte ii
 chico norte ii
 molinos norte
 chico norte ii
 urbanizacion santa coloma
 chico norte
 chico norte iii
 antiguo country
 bella suiza
 el retiro
 chico norte ii
 la veracruz
 la calleja
 el refugio
 chico
 chico norte ii
 chico
 chico norte iii
 chico norte ii
 las acacias
 santa bibiana
 emaus
 santa bibiana
 los rosales
 santa bibiana
 santa barbara occidental
 chico
 chico
 molinos norte
 chico norte ii
 chico
 san patricio
 santa bibiana
 chico norte
 el vergel occidental
 antiguo country
 chico norte iii
 chico norte ii
 chico norte iii
 el refugio
 bellavista
 santa bibiana

 el batan
 chico norte ii
 chico norte ii
 santa barbara occidental
 chico norte ii
 chico norte
 chico norte ii
 rincon del chico
 el nogal
 la cabrera
 molinos norte
 chico
 molinos norte
 la carolina
 chico norte
 san patricio
 la salle
 chico
 los rosales
 chico
 santa bibiana
 chico norte iii
 san patricio
 chico norte iii
 chico
 santa bibiana
 antigu

 chico norte ii
 los rosales
 chico norte iii
 la carolina
 molinos norte
 el nogal
 bosque izquierdo
 santa barbara occidental
 chico norte
 chico norte iii
 la cabrera
 santa barbara alta
 el nogal
 chico norte ii
 san patricio
 chico
 chico norte iii
 ingemar
 el nogal
 santa barbara alta
 chico norte
 chico norte iii
 chico norte ii
 chico norte iii
 santa barbara occidental
 chico norte
 emaus
 la carolina
 lago gaitan
 santa ana occidental
 el retiro
 samper
 chico norte iii
 antiguo country
 chico
 chico
 santa barbara occidental
 santa barbara occidental
 santa bibiana
 santa bibiana
 bellavista
 chico norte iii
 chico norte ii
 chico norte ii
 chico norte ii
 los rosales
 chico
 chico norte ii
 la cabrera
 santa ana occidental
 chico
 chico norte
 la cabrera
 antiguo country
 emaus
 rincon del chico
 emaus
 emaus
 chico
 chico
 chico norte iii
 santa bibiana
 molinos norte
 emaus
 santa bibiana
 chico norte ii
 el retiro
 la veracruz
 la calleja
 el refugio
 chico norte ii
 sa

In [76]:
print(df.shape[0])
print(cont)

15631
15631


In [51]:
df_upz

,UPlCodigo,UPlTipo,UPlNombre,UPlArea,pro_location
0,UPZ100,1,GALERIAS,2.375681e+06,palermo
1,UPZ100,1,GALERIAS,2.375681e+06,belalcazar
2,UPZ100,1,GALERIAS,2.375681e+06,sears
3,UPZ100,1,GALERIAS,2.375681e+06,galerias
4,UPZ100,1,GALERIAS,2.375681e+06,banco central
...,...,...,...,...,...
463,UPZ42,1,VENECIA,6.638420e+06,nuevo muzu
464,UPZ42,1,VENECIA,6.638420e+06,el condor
465,UPZ42,1,VENECIA,6.638420e+06,laguneta
466,UPZ52,1,LA FLORA,1.877160e+06,villa maria


In [ ]:
df